Open Play Shots - Expected Behind Model - Model Tuning and Building - GBM

In [65]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")

In [66]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_score_model.config import open_shots_file_path
# from expected_score_model.domain.preprocessing.data_preprocessor import DataPreprocessor
# from expected_score_model.domain.contracts.mappings import Mappings
from expected_score_model.domain.contracts.modelling_data_contract import ModellingDataContract
from expected_score_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_score_model.domain.modelling.supermodel import SuperXGBClassifier
from expected_score_model.domain.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Model Version

In [67]:
model_version = 5
model_name = 'expected_behind_open'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/"

In [68]:
RESPONSE = ModellingDataContract.RESPONSE_BEHIND

In [69]:
FEATURES = ModellingDataContract.open_goal_modelling_feature_list

In [70]:
MONOTONE_CONSTRAINTS = ModellingDataContract.monotone_constraints_open_goal

Load Data

In [71]:
df_modelling = pd.read_csv(open_shots_file_path)
df_modelling.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,GoalTrainingSet,GoalTestSet,GoalValidationSet,BehindTrainingSet,BehindTestSet,BehindValidationSet,MissTrainingSet,MissTestSet,MissValidationSet
9211,211,kickIn,behind,1705.0,4,744,775.0,Sydney,Sydney,Tom Papley,Tom_Papley,Kick,56.0,-36.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,0,1,0,1,0,1,107.0,32.0,75.0,Kick,Gather,Kick,Handball Received,56.0,48.0,7.0,4.0,-36.0,-44.0,-47.0,-37.0,775.0,771.0,769.0,766.0,31.0,4.0,11.313708,24.0,36.0,43.266615,0.982794,56.309932,0.082503,4.727050,0.258900,14.833859,576.0,3.178054,False,True,False,False,True,False,False,True,False,False
9212,212,kickIn,goal,1740.0,4,787,900.0,Geelong,Geelong,Brandan Parfitt,Brandan_Parfitt,Kick,75.0,0.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,113.0,33.0,80.0,Kick,Handball Received,Handball,Loose Ball Get,75.0,75.0,76.0,75.0,0.0,-1.0,-5.0,-6.0,900.0,899.0,898.0,897.0,113.0,1.0,1.000000,5.0,0.0,5.000000,0.000000,0.000000,2.168022,124.218492,-1.429422,-81.899864,25.0,1.609438,False,False,False,True,False,False,True,True,False,False
9213,218,possGain,goal,1837.0,4,1136,1198.0,Geelong,Geelong,Sam De Koning,Sam_De_Koning,Kick,78.0,1.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,119.0,39.0,80.0,Kick,Gather,Kick,Handball Received,78.0,78.0,65.0,64.0,1.0,1.0,-12.0,-9.0,1198.0,1193.0,1190.0,1188.0,62.0,5.0,0.000000,2.0,1.0,2.236068,0.463648,26.565051,-2.442748,-139.959156,-0.440569,-25.242748,4.0,0.693147,False,True,False,False,True,False,False,True,False,False
9214,227,throwIn,goal,1901.0,4,1452,1458.0,Geelong,Geelong,Joel Selwood,Joel_Selwood,Kick,37.0,17.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,125.0,45.0,80.0,Kick,Gather,Knock On,Handball,37.0,37.0,43.0,34.0,17.0,19.0,20.0,25.0,1458.0,1457.0,1455.0,1453.0,6.0,1.0,2.000000,43.0,17.0,46.238512,0.376490,21.571307,0.129338,7.410516,0.403551,23.121747,1849.0,3.761200,False,False,False,True,True,False,False,True,False,False
9215,233,possGain,goal,1971.0,4,1744,1753.0,Sydney,Sydney,Tom Papley,Tom_Papley,Kick,48.0,-20.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,0,6,137.0,45.0,92.0,Kick,Bounce,Handball Received,Handball,48.0,38.0,35.0,37.0,-20.0,-31.0,-33.0,-37.0,1753.0,1752.0,1748.0,1747.0,9.0,1.0,14.866069,32.0,20.0,37.735925,0.558599,32.005383,0.144862,8.299977,0.461317,26.431498,1024.0,3.465736,False,True,False,False,False,False,True,True,False,False


In [72]:
df_modelling[['ballUp', 'centreBounce', 'kickIn', 'possGain', 'throwIn']] = pd.get_dummies(df_modelling['Initial_State'])

In [73]:
training_data = df_modelling[df_modelling[RESPONSE+"TrainingSet"]]
test_data = df_modelling[df_modelling[RESPONSE+"TestSet"]]
cal_data = df_modelling[df_modelling[RESPONSE+"ValidationSet"]]

In [74]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]
X_cal, y_cal = cal_data.drop(columns=[RESPONSE]), cal_data[RESPONSE]

In [75]:
X_test.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,GoalTrainingSet,GoalTestSet,GoalValidationSet,BehindTrainingSet,BehindTestSet,BehindValidationSet,MissTrainingSet,MissTestSet,MissValidationSet,ballUp,centreBounce,kickIn,possGain,throwIn
5,43,possGain,behind,444.0,1,1272,1317.0,Brisbane Lions,Brisbane Lions,Joe Daniher,Joe_Daniher,Kick,27.0,31.0,ineffective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,0,1,1,0,18.0,9.0,9.0,Kick,Handball Received,Handball,Free For,27.0,28.0,34.0,35.0,31.0,34.0,35.0,34.0,1317.0,1316.0,1314.0,1308.0,45.0,1.0,3.162278,51.0,31.0,59.682493,0.546167,31.293039,0.091898,5.265373,0.282203,16.169053,2601.0,3.931826,False,False,False,True,False,True,False,True,False,False,0,0,0,1,0
6,46,possGain,goal,476.0,1,1383,1389.0,Sydney,Sydney,Isaac Heeney,Isaac_Heeney,Kick,56.0,6.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,0,6,19.0,9.0,10.0,Kick,Handball Received,Handball,Handball Received,56.0,58.0,58.0,58.0,6.0,21.0,25.0,26.0,1389.0,1389.0,1388.0,1387.0,6.0,0.0,15.132746,22.0,6.0,22.803509,0.266252,15.255119,0.276208,15.825576,0.987285,56.567262,484.0,3.091042,False,True,False,False,False,True,False,False,True,False,0,0,0,1,0
8,79,possGain,goal,747.0,2,709,720.0,Sydney,Sydney,Sam Wicks,Sam_Wicks,Kick,41.0,18.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,0,6,20.0,40.0,-20.0,Kick,Handball Received,Handball,Handball Received,41.0,38.0,37.0,36.0,18.0,16.0,15.0,15.0,720.0,720.0,719.0,718.0,11.0,0.0,3.605551,37.0,18.0,41.146081,0.452778,25.942295,0.140721,8.062731,0.443767,25.425977,1369.0,3.610918,False,True,False,False,False,True,False,False,True,False,0,0,0,1,0
9,83,throwIn,goal,781.0,2,848,861.0,Brisbane Lions,Brisbane Lions,Jarryd Lyons,Jarryd_Lyons,Kick,46.0,2.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,6,0,20.0,46.0,-26.0,Kick,Gather,Knock On,Handball,46.0,41.0,35.0,30.0,2.0,2.0,-1.0,-2.0,861.0,861.0,860.0,858.0,13.0,0.0,5.000000,32.0,2.0,32.062439,0.062419,3.576334,0.201226,11.529413,0.656522,37.615967,1024.0,3.465736,False,True,False,False,False,True,False,True,False,False,0,0,0,0,1
12,95,possGain,goal,888.0,2,1285,1306.0,Brisbane Lions,Brisbane Lions,Keidean Coleman,Keidean_Coleman,Kick,48.0,0.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,6,0,32.0,46.0,-14.0,Kick,Loose Ball Get Crumb,Kick Inside 50 Result,Kick Into F50,48.0,45.0,51.0,31.0,0.0,-5.0,-8.0,-43.0,1306.0,1305.0,1303.0,1303.0,21.0,1.0,5.830952,30.0,0.0,30.000000,0.000000,0.000000,0.215789,12.363772,0.713012,40.852606,900.0,3.401197,False,True,False,False,False,True,False,True,False,False,0,0,0,1,0


Preprocess Data

In [76]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [88]:
X_train_preproc = X_train[FEATURES]
X_test_preproc = X_test[FEATURES]
X_cal_preproc = X_cal[FEATURES]

In [78]:
X_train_preproc.shape, X_test_preproc.shape

((5897, 17), (1844, 17))

In [79]:
X_train_preproc.head()

,x0,x1,x2,x3,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,Distance_to_Right_Goal_x,Distance_to_Middle_y,Chain_Duration,Time_Since_Last_Action
0,26.0,18.0,11.0,6.0,8.246211,56.080300,0.383818,0.106164,0,1,0,0,0,52.0,21.0,26.0,1.0
1,35.0,28.0,25.0,22.0,7.000000,47.010637,0.416065,0.125105,0,0,0,1,0,43.0,19.0,19.0,1.0
3,34.0,19.0,10.0,7.0,15.132746,46.486557,0.328553,0.130930,0,0,0,1,0,44.0,15.0,13.0,2.0
4,40.0,36.0,34.0,32.0,4.123106,39.560081,0.281772,0.156423,0,0,0,1,0,38.0,11.0,5.0,3.0
7,65.0,62.0,60.0,60.0,3.162278,19.849433,0.856706,0.216802,0,0,0,1,0,13.0,15.0,38.0,1.0


In [80]:
X_test_preproc.head()

,x0,x1,x2,x3,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,Distance_to_Right_Goal_x,Distance_to_Middle_y,Chain_Duration,Time_Since_Last_Action
5,27.0,28.0,34.0,35.0,3.162278,59.682493,0.546167,0.091898,0,0,0,1,0,51.0,31.0,45.0,1.0
6,56.0,58.0,58.0,58.0,15.132746,22.803509,0.266252,0.276208,0,0,0,1,0,22.0,6.0,6.0,0.0
8,41.0,38.0,37.0,36.0,3.605551,41.146081,0.452778,0.140721,0,0,0,1,0,37.0,18.0,11.0,0.0
9,46.0,41.0,35.0,30.0,5.000000,32.062439,0.062419,0.201226,0,0,0,0,1,32.0,2.0,13.0,0.0
12,48.0,45.0,51.0,31.0,5.830952,30.000000,0.000000,0.215789,0,0,0,1,0,30.0,0.0,21.0,1.0


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [81]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, monotonicity_constraints=MONOTONE_CONSTRAINTS)

In [82]:
xgb_tuner.training_data.head()

,x0,x1,x2,x3,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,Distance_to_Right_Goal_x,Distance_to_Middle_y,Chain_Duration,Time_Since_Last_Action
0,26.0,18.0,11.0,6.0,8.246211,56.080300,0.383818,0.106164,0,1,0,0,0,52.0,21.0,26.0,1.0
1,35.0,28.0,25.0,22.0,7.000000,47.010637,0.416065,0.125105,0,0,0,1,0,43.0,19.0,19.0,1.0
3,34.0,19.0,10.0,7.0,15.132746,46.486557,0.328553,0.130930,0,0,0,1,0,44.0,15.0,13.0,2.0
4,40.0,36.0,34.0,32.0,4.123106,39.560081,0.281772,0.156423,0,0,0,1,0,38.0,11.0,5.0,3.0
7,65.0,62.0,60.0,60.0,3.162278,19.849433,0.856706,0.216802,0,0,0,1,0,13.0,15.0,38.0,1.0


In [83]:
xgb_tuner.tune_hyperparameters()

[I 2023-07-11 13:50:03,072] A new study created in memory with name: no-name-37c6bd14-ca9f-4adc-b9b0-ed8193d494be
[I 2023-07-11 13:50:03,151] Trial 0 finished with value: 0.6502564707716285 and parameters: {'max_depth': 15, 'min_child_weight': 68, 'eta': 0.12770818209441928, 'gamma': 0.0015258126769293143, 'lambda': 0.0062080592387658414, 'alpha': 8.377688147005154, 'subsample': 0.31056932305591745, 'colsample_bytree': 0.4369128212717649}. Best is trial 0 with value: 0.6502564707716285.
[I 2023-07-11 13:50:03,188] Trial 1 finished with value: 0.6714736048668415 and parameters: {'max_depth': 7, 'min_child_weight': 8, 'eta': 0.025184824306393693, 'gamma': 0.8239703645442905, 'lambda': 0.03650149299197544, 'alpha': 0.005135410289025483, 'subsample': 0.649310085897579, 'colsample_bytree': 0.4752757512309769}. Best is trial 0 with value: 0.6502564707716285.
[I 2023-07-11 13:50:03,210] Trial 2 finished with value: 0.6454910768831363 and parameters: {'max_depth': 13, 'min_child_weight': 75, '

Number of finished trials:  1000
Best trial:
  Value: 0.6212565623302377
  Params: 
    max_depth: 9
    min_child_weight: 99
    eta: 0.4519457919700433
    gamma: 0.08875401321523607
    lambda: 0.0058401692982087685
    alpha: 0.000918803548530007
    subsample: 0.7181371963717417
    colsample_bytree: 0.636601947872166


In [84]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 9,
 'min_child_weight': 99,
 'eta': 0.4519457919700433,
 'gamma': 0.08875401321523607,
 'lambda': 0.0058401692982087685,
 'alpha': 0.000918803548530007,
 'subsample': 0.7181371963717417,
 'colsample_bytree': 0.636601947872166}

Training Model - SuperXGBClassifier class for training and predictions

In [85]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = MONOTONE_CONSTRAINTS

In [89]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test,
                               X_cal = X_cal_preproc,
                               y_cal = y_cal,
                               params = params)

In [90]:
super_xgb.fit()

[0]	validation_0-logloss:0.66216	validation_1-logloss:0.66280
[1]	validation_0-logloss:0.65124	validation_1-logloss:0.65468
[2]	validation_0-logloss:0.64740	validation_1-logloss:0.65201
[3]	validation_0-logloss:0.64138	validation_1-logloss:0.64493
[4]	validation_0-logloss:0.63915	validation_1-logloss:0.64521
[5]	validation_0-logloss:0.63601	validation_1-logloss:0.64219
[6]	validation_0-logloss:0.63509	validation_1-logloss:0.64191
[7]	validation_0-logloss:0.63376	validation_1-logloss:0.64158
[8]	validation_0-logloss:0.63208	validation_1-logloss:0.64096
[9]	validation_0-logloss:0.63094	validation_1-logloss:0.64136
[10]	validation_0-logloss:0.63011	validation_1-logloss:0.64119
[11]	validation_0-logloss:0.62944	validation_1-logloss:0.64127
[12]	validation_0-logloss:0.62827	validation_1-logloss:0.63961
[13]	validation_0-logloss:0.62749	validation_1-logloss:0.63918
[14]	validation_0-logloss:0.62726	validation_1-logloss:0.64071
[15]	validation_0-logloss:0.62674	validation_1-logloss:0.64050
[1

In [91]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.636601947872166, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.08875401321523607, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.45194...
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=99, missing=nan,
              monotone_constraints={'Angle_to_Middle_Goal': -1,
                                    'Distance_Since_Last_Action': 1,
                                    'Distance_to_Middle_Goal': -1,
                                    'Visible_Goal_Angle': 1, 'x0': -1},
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [92]:
super_xgb.xgb_model.get_booster().feature_names

['x0',
 'x1',
 'x2',
 'x3',
 'Distance_Since_Last_Action',
 'Distance_to_Middle_Goal',
 'Angle_to_Middle_Goal',
 'Visible_Goal_Angle',
 'ballUp',
 'centreBounce',
 'kickIn',
 'possGain',
 'throwIn',
 'Distance_to_Right_Goal_x',
 'Distance_to_Middle_y',
 'Chain_Duration',
 'Time_Since_Last_Action']

In [93]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [96]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]
cal_probas = super_xgb.predict_proba(X_cal_preproc)[:, 1]

In [97]:
super_xgb.calibrate()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [98]:
train_cal_probas = super_xgb.predict_proba(X_train_preproc, calibrate=True)
test_cal_probas = super_xgb.predict_proba(X_test_preproc, calibrate=True)

Check Average Predictions

In [99]:
train_probas.mean(), training_data[RESPONSE].mean(), train_cal_probas.mean()

(0.3685548, 0.3647617432592844, 0.36857276339947526)

In [100]:
test_probas.mean(), test_data[RESPONSE].mean(), test_cal_probas.mean()

(0.36439252, 0.36496746203904556, 0.3649869587985149)

Export model

In [101]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [102]:
train_info = training_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
train_export = pd.concat([train_info, X_train_preproc.reset_index(drop=True)], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_probas'] = train_probas
train_export['xgb_probas_cal'] = train_cal_probas
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,Match_ID,Chain_Number,Order,Behind,x0,x1,x2,x3,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,Distance_to_Right_Goal_x,Distance_to_Middle_y,Chain_Duration,Time_Since_Last_Action,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,1,11.0,0,26.0,18.0,11.0,6.0,8.246211,56.080300,0.383818,0.106164,0,1,0,0,0,52.0,21.0,26.0,1.0,1,0.538129,0.493594
1,202101_BrisbaneLions_Sydney,6,61.0,0,35.0,28.0,25.0,22.0,7.000000,47.010637,0.416065,0.125105,0,0,0,1,0,43.0,19.0,19.0,1.0,0,0.369051,0.375746
2,202101_BrisbaneLions_Sydney,21,222.0,0,34.0,19.0,10.0,7.0,15.132746,46.486557,0.328553,0.130930,0,0,0,1,0,44.0,15.0,13.0,2.0,0,0.486473,0.461428
3,202101_BrisbaneLions_Sydney,38,392.0,0,40.0,36.0,34.0,32.0,4.123106,39.560081,0.281772,0.156423,0,0,0,1,0,38.0,11.0,5.0,3.0,0,0.415615,0.411981
4,202101_BrisbaneLions_Sydney,50,512.0,1,65.0,62.0,60.0,60.0,3.162278,19.849433,0.856706,0.216802,0,0,0,1,0,13.0,15.0,38.0,1.0,0,0.424042,0.418209


In [103]:
test_info = test_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
test_export = pd.concat([test_info, X_test_preproc.reset_index(drop=True)], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_probas'] = test_probas
test_export['xgb_probas_cal'] = test_cal_probas
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,Match_ID,Chain_Number,Order,Behind,x0,x1,x2,x3,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,Distance_to_Right_Goal_x,Distance_to_Middle_y,Chain_Duration,Time_Since_Last_Action,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,43,444.0,1,27.0,28.0,34.0,35.0,3.162278,59.682493,0.546167,0.091898,0,0,0,1,0,51.0,31.0,45.0,1.0,1,0.527646,0.487311
1,202101_BrisbaneLions_Sydney,46,476.0,0,56.0,58.0,58.0,58.0,15.132746,22.803509,0.266252,0.276208,0,0,0,1,0,22.0,6.0,6.0,0.0,0,0.311128,0.326109
2,202101_BrisbaneLions_Sydney,79,747.0,0,41.0,38.0,37.0,36.0,3.605551,41.146081,0.452778,0.140721,0,0,0,1,0,37.0,18.0,11.0,0.0,0,0.378141,0.383068
3,202101_BrisbaneLions_Sydney,83,781.0,0,46.0,41.0,35.0,30.0,5.000000,32.062439,0.062419,0.201226,0,0,0,0,1,32.0,2.0,13.0,0.0,0,0.363029,0.370827
4,202101_BrisbaneLions_Sydney,95,888.0,0,48.0,45.0,51.0,31.0,5.830952,30.000000,0.000000,0.215789,0,0,0,1,0,30.0,0.0,21.0,1.0,0,0.298751,0.314812


Save preprocessor

In [ ]:
# joblib.dump(preprocessor, preprocessor_output_path + preprocessor_file_name + ".joblib")